### **Imports**

In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
# from scipy.sparse import random
# from scipy import stats
# import seaborn as sns
import matplotlib.pyplot as plt
# import plotly.express as px
# from sklearn.model_selection import train_test_split
import geopandas as gpd
import shapely
import shapely.wkt
from geomet import wkt

In [3]:
dfCapi= pd.read_csv("parte1.csv")
barrios=pd.read_csv("barrios.csv")

In [4]:
#Armamos una pivot con los place_name de nuestro data frame para checkear si matchean con el csv barrios
dfCapi_grouped=dfCapi.groupby('place_name').mean()
dfCapi_grouped

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,rooms_reg,price_m2_sin_out
place_name,,,,,,,,,,,,,,,,
Abasto,57068.425287,NaN,-34.603574,-58.411650,153900.178571,2.405393e+06,136325.355595,68.085714,56.468354,2133.465234,2684.017722,5.583333,2.591837,1841.291667,2.371795,2136.905318
Agronomía,48771.520000,3436487.0,-34.591886,-58.485309,137690.434783,2.429479e+06,137690.434783,57.157895,56.130435,2257.803667,2549.937271,6.000000,2.476190,3466.666667,2.480000,2177.139255
Almagro,58316.205078,3436397.0,-34.607978,-58.421654,198033.862950,2.465427e+06,139727.767782,65.890679,65.024465,2227.575442,4116.568548,4.325581,2.415033,2230.449612,2.285057,2211.739443
Balvanera,58719.285476,6693228.0,-34.609654,-58.402777,138461.352734,2.265464e+06,128394.909630,71.047085,73.956757,1952.816681,2449.065762,6.072581,2.690722,2119.862275,2.588235,1938.858732
Barracas,49487.403270,3436134.0,-34.637722,-58.374475,161649.898017,2.751382e+06,155934.257365,93.158879,83.514970,1949.716241,2303.013886,14.666667,2.650000,2343.377358,2.569579,1921.320786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Riachuelo,119715.000000,3427379.0,-34.690946,-58.469362,165000.000000,2.911342e+06,165000.000000,NaN,20.000000,NaN,8250.000000,NaN,4.000000,NaN,4.000000,1229.508197
Villa Santa Rita,39210.215686,6693230.0,-34.615546,-58.479948,163551.000000,2.885776e+06,163551.000000,113.409091,60.714286,1883.887256,28893.221143,5.300000,2.540541,800.000000,2.595745,1908.348546
Villa Soldati,53242.750000,3427366.0,-34.663860,-58.447185,201125.000000,1.487702e+06,84315.355000,88.000000,90.000000,784.090909,2894.332216,6.000000,4.000000,NaN,4.000000,844.893366


In [5]:
geoprop = gpd.GeoDataFrame(dfCapi, geometry=gpd.points_from_xy(dfCapi.lon, dfCapi.lat))
barrios["WKT_shapely"] = barrios["WKT"].apply(shapely.wkt.loads) 
geo_barrios = gpd.GeoDataFrame(barrios, geometry='WKT_shapely')

In [10]:
# Funcion para obtener el BARRIO segun las coordenadas geográficas de la propiedad
def barrio_gpd(x):
    if ((x.place_name == "Capital Federal") & (np.isnan(x.lat) == False)): # Solo busco los que tienen place_name = Capital Federal y valores de lat-lon
        for barrio in geo_barrios.barrio:
            r = geo_barrios[geo_barrios.barrio == barrio].contains(x.geometry).values[0] #Itero sobre todos los barrios y me fijo cual contiene al punto
            if (r):
                return barrio #Devuelvo el nombre del barrio que contiene el punto
    else: 
        return x.place_name # Si place_name != Capital Federal o no tenia coordenadas devuelvo el valor original

In [11]:
# Reemplazo el barrio por gps en place_name
dfCapi.loc[:,"place_name"] = geoprop.apply(barrio_gpd, axis=1)

In [13]:
import geopy.distance
df_subte = pd.read_csv("./estaciones-de-subte.csv", sep=',')
geo_subte = gpd.GeoDataFrame(df_subte,geometry=gpd.points_from_xy(df_subte.long,df_subte.lat))
geoprop.loc[:,"distancia_subte"] = 10000 #seteo un valor por defecto para poder empezar comparando

In [14]:
for i in geo_subte.index: #itero sobre cada estacion
    for j in geoprop.index: #itero sobre cada casa
        if (geoprop.iloc[j].geometry.x == geoprop.iloc[j].geometry.x): #condicion rara para chequear si el valor no es None
            dist = geopy.distance.geodesic((geo_subte.loc[i,"geometry"].x,geo_subte.loc[i,"geometry"].y),
                                           (geoprop.loc[j,'geometry'].x,geoprop.loc[j,'geometry'].y)).meters # calculo la distancia
            if (dist < geoprop.distancia_subte[j]): #chequeo que sea menor a la registrada
                geoprop.loc[j, 'distancia_subte'] = dist #si lo es, le digo que esa sea la nueva distancia

In [15]:
geoprop.distancia_subte.replace(10000,np.NaN, inplace=True) #reemplazo los casos que quedaron con el valor inicial arbitrario por null
geoprop.distancia_subte = geoprop.distancia_subte.round(2)
dfCapi["distancia_subte"] = geoprop.distancia_subte

In [16]:
dfCapi.to_csv("parte2.csv")
barrios.to_csv("barrios.csv")